In [ ]:
!pip install --quiet img2vec_pytorch    # görüntüleri vektör temsillerine dönüştürmek için, sebebi: görüntüdeki her bir pikselin RGB değerlerini önce binary haline convert edip sonra her bir pikselin binary değerlerinin sayısal karşılığını tek bir vektorde tutuyor
!pip install --quiet arrow               #tarih ve zaman işlemleri için
!pip install umap-learn                  # vektör temsillerini daha düşük boyutlu bir uzaya projeksiyonlar. Daha sonra, bu düşük boyutlu uzayda görselleştirmek için iki boyutlu koordinatlara indirgenmiş olan veriyi içeren bir DataFrame oluşturur.

In [ ]:
from warnings import filterwarnings
filterwarnings(action='ignore', category=FutureWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from img2vec_pytorch import Img2Vec
from PIL import Image                  #PIL, resim dosyalarını açma, kaydetme, dönüştürme ve işleme gibi bir dizi işlevi içerir.
from arrow import now
from glob import glob             #Örneğin, "*.txt" deseni, bir dizindeki tüm .txt uzantılı dosyaları eşleştirir.
import pandas as pd                     # Veri analizi ve manipülasyonu için
from os.path import basename             #bir dosya yolundan dosya adını almak için kullanılır.

img2vec = Img2Vec(cuda=False, model='resnet-18', layer='default', layer_output_size=512)     #Img2Vec sınıfından bir nesne oluşturuyoruz. Bu nesne, görüntülerin vektör temsillerini elde etmek için kullanılacak. ResNet de CNN ailesine ait bir modeldir.

GLOB_Benign = '/content/drive/MyDrive/melonamacancer/train/Benign/*'
GLOB_Malignant = '/content/drive/MyDrive/melonamacancer/train/Malignant/*'

def get_from_glob(arg: str, tag: str) -> list:                  #arg, işlenecek dosyaların yolu ve tag, her resmin hangi kategoriye ait olduğu
    result = []
    for input_file in glob(pathname=arg):                        #glob ile belirtilen dosya yolundaki (ai veya real klasörleri) her bir dosya işleme girer sırasıyla
        name = basename(input_file)
        try:
            with Image.open(fp=input_file, mode='r') as image:
                vector = img2vec.get_vec(image, tensor=True).numpy().reshape(512,)
                result.append(pd.Series(data=[tag, name, vector], index=['tag', 'name', 'value']))
        except RuntimeError:
            # we only have a few failures so we're just going to discard them
            pass
    return result

time_start = now()
Benign = get_from_glob(arg=GLOB_Benign, tag='benign')
print('done encoding the AI images in {}'.format(now() - time_start))
Malignant = get_from_glob(arg=GLOB_Malignant, tag='malignant')
df = pd.DataFrame(data=Benign + Malignant)
print('done in {}'.format(now() - time_start))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 72.0MB/s]


done encoding the AI images in 0:14:00.449821
done in 0:26:00.166228


In [ ]:
df

,tag,name,value
0,benign,5765.jpg,"[1.85632, 0.8769164, 0.0, 2.4691164, 0.0456506..."
1,benign,5764.jpg,"[0.8542326, 0.010712791, 0.0, 1.8745635, 0.0, ..."
2,benign,5768.jpg,"[1.0994872, 0.13306834, 0.052660577, 1.2007022..."
3,benign,5766.jpg,"[0.44033393, 0.004941154, 0.22826417, 0.623123..."
4,benign,5767.jpg,"[1.6859809, 0.3467841, 0.065607, 0.8831522, 0...."
...,...,...,...
11925,malignant,1569.jpg,"[0.7718353, 0.06514394, 0.06920192, 1.6364119,..."
11926,malignant,1573.jpg,"[2.5451112, 0.6679669, 0.27408683, 1.0507066, ..."
11927,malignant,1584.jpg,"[0.96708316, 0.28628924, 0.3423002, 1.0374732,..."
11928,malignant,1590.jpg,"[1.7722467, 0.4236938, 0.35403943, 0.83825487,..."


In [ ]:
from umap import UMAP
umap = UMAP(random_state=2024, verbose=True, n_jobs=1,)
model_df = pd.DataFrame(data=umap.fit_transform(X=df['value'].apply(pd.Series)), columns=['x', 'y'])            #value değerlerinin 2D e indirgenmiş hallerini sadece grafikte datapoint lerin dağılımını görmede kuallanıcaz
plot_df = pd.concat(objs=[df, model_df], axis=1)                                                                # modeli eğitirken 2D değerlerini değil normal herbir image için elde edilmiş vector değerlerini kullanacağız
print('done with UMAP in {}'.format(now() - time_start))

UMAP(n_jobs=1, random_state=2024, verbose=True)
Thu Apr 25 09:59:55 2024 Construct fuzzy simplicial set
Thu Apr 25 09:59:55 2024 Finding Nearest Neighbors
Thu Apr 25 09:59:55 2024 Building RP forest with 10 trees
Thu Apr 25 10:00:03 2024 NN descent for 14 iterations
	 1  /  14
	 2  /  14
	 3  /  14
	 4  /  14
	 5  /  14
	Stopping threshold met -- exiting after 5 iterations
Thu Apr 25 10:00:22 2024 Finished Nearest Neighbor Search
Thu Apr 25 10:00:27 2024 Construct embedding


Epochs completed:   0%|            0/200 [00:00]

	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Thu Apr 25 10:00:38 2024 Finished embedding
done with UMAP in 0:32:56.270836


In [ ]:
print(model_df)

              x         y
0     -0.932643 -1.825974
1     -1.778681 -2.429216
2      4.960999 -1.497431
3      4.451528 -2.442977
4     -3.887175  0.836020
...         ...       ...
11925  3.635844 -0.548214
11926  0.524525  0.655612
11927 -0.328305  8.833584
11928 -0.541048  3.065094
11929  0.075235  2.802912

[11930 rows x 2 columns]


In [ ]:
print(plot_df)

             tag      name                                              value  \
0         benign  5765.jpg  [1.85632, 0.8769164, 0.0, 2.4691164, 0.0456506...   
1         benign  5764.jpg  [0.8542326, 0.010712791, 0.0, 1.8745635, 0.0, ...   
2         benign  5768.jpg  [1.0994872, 0.13306834, 0.052660577, 1.2007022...   
3         benign  5766.jpg  [0.44033393, 0.004941154, 0.22826417, 0.623123...   
4         benign  5767.jpg  [1.6859809, 0.3467841, 0.065607, 0.8831522, 0....   
...          ...       ...                                                ...   
11925  malignant  1569.jpg  [0.7718353, 0.06514394, 0.06920192, 1.6364119,...   
11926  malignant  1573.jpg  [2.5451112, 0.6679669, 0.27408683, 1.0507066, ...   
11927  malignant  1584.jpg  [0.96708316, 0.28628924, 0.3423002, 1.0374732,...   
11928  malignant  1590.jpg  [1.7722467, 0.4236938, 0.35403943, 0.83825487,...   
11929  malignant  1589.jpg  [0.16221376, 1.4595754, 0.21976739, 0.19024402...   

              x         y  

In [ ]:
from plotly.express import scatter
scatter(data_frame=plot_df, x='x', y='y', color='tag', hover_name='name', height=900)

In [ ]:
img2vec = Img2Vec(cuda=False, model='resnet-18', layer='default', layer_output_size=512)     #Img2Vec sınıfından bir nesne oluşturuyoruz. Bu nesne, görüntülerin vektör temsillerini elde etmek için kullanılacak. ResNet de CNN ailesine ait bir modeldir.

GLOB_Benign_test = '/content/drive/MyDrive/melonamacancer/test/Benign/*'
GLOB_Malignant_test = '/content/drive/MyDrive/melonamacancer/test/Malignant/*'

def get_from_glob(arg: str, tag: str) -> list:                  #arg, işlenecek dosyaların yolu ve tag, her resmin hangi kategoriye ait olduğu
    result = []
    for input_file in glob(pathname=arg):                        #glob ile belirtilen dosya yolundaki (ai veya real klasörleri) her bir dosya işleme girer sırasıyla
        name = basename(input_file)
        try:
            with Image.open(fp=input_file, mode='r') as image:
                vector = img2vec.get_vec(image, tensor=True).numpy().reshape(512,)
                result.append(pd.Series(data=[tag, name, vector], index=['tag', 'name', 'value']))
        except RuntimeError:
            # we only have a few failures so we're just going to discard them
            pass
    return result

time_start = now()
Benign_test = get_from_glob(arg=GLOB_Benign_test, tag='benign')
print('done encoding the images in {}'.format(now() - time_start))
Malignant_test = get_from_glob(arg=GLOB_Malignant_test, tag='malignant')
df_test = pd.DataFrame(data=Benign_test + Malignant_test)
print('done in {}'.format(now() - time_start))

X_test=df_test['value'].apply(pd.Series)
y_test=df_test['tag']

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.



done encoding the images in 0:06:22.923793
done in 0:08:35.246068


In [ ]:
#test setini görselleştirme
umap = UMAP(random_state=2024, verbose=True, n_jobs=1,)
model_df_test = pd.DataFrame(data=umap.fit_transform(X=df_test['value'].apply(pd.Series)), columns=['x', 'y'])            #value değerlerinin 2D e indirgenmiş hallerini sadece grafikte datapoint lerin dağılımını görmede kuallanıcaz
plot_df_test = pd.concat(objs=[df_test, model_df_test], axis=1)                                                                # modeli eğitirken 2D değerlerini değil normal herbir image için elde edilmiş vector değerlerini kullanacağız
print('done with UMAP in {}'.format(now() - time_start))

UMAP(n_jobs=1, random_state=2024, verbose=True)
Thu Apr 25 10:41:20 2024 Construct fuzzy simplicial set
Thu Apr 25 10:41:23 2024 Finding Nearest Neighbors
Thu Apr 25 10:41:28 2024 Finished Nearest Neighbor Search
Thu Apr 25 10:41:28 2024 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Thu Apr 25 10:41:33 2024 Finished embedding
done with UMAP in 0:08:56.510750


In [ ]:
scatter(data_frame=plot_df_test, x='x', y='y', color='tag', hover_name='name', height=900)

In [ ]:
df_test

,tag,name,value
0,benign,6299.jpg,"[0.3909477, 0.85957646, 0.1606896, 1.1681563, ..."
1,benign,6300.jpg,"[0.5516756, 0.040642407, 0.07764559, 2.038202,..."
2,benign,6301.jpg,"[1.6560903, 0.5901034, 0.051712174, 1.2118051,..."
3,benign,6311.jpg,"[2.4047737, 0.4185635, 0.014620303, 0.7268153,..."
4,benign,6312.jpg,"[0.20154692, 1.577353, 0.0, 1.6303614, 0.48096..."
...,...,...,...
2015,malignant,5612.jpg,"[0.6409828, 0.38275692, 0.08618814, 0.44692302..."
2016,malignant,5613.jpg,"[3.3030813, 0.028153513, 0.1810992, 2.0821433,..."
2017,malignant,5608.jpg,"[1.6141049, 0.7547744, 0.08927266, 0.72970736,..."
2018,malignant,5621 (1).jpg,"[1.9083118, 0.38536686, 0.93993336, 0.5957861,..."


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train=df['value'].apply(pd.Series)
y_train=df['tag']

model = LogisticRegression(max_iter=100000)
model.fit(X_train, y_train)


print('accuracy: {:5.2f} pct'.format(100 * accuracy_score(y_test, model.predict(X_test))))

accuracy: 88.47 pct


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true = y_test, y_pred = model.predict(X_test)))

              precision    recall  f1-score   support

      benign       0.85      0.93      0.89      1000
   malignant       0.92      0.84      0.88      1020

    accuracy                           0.88      2020
   macro avg       0.89      0.89      0.88      2020
weighted avg       0.89      0.88      0.88      2020



ANN İLE MODEL EĞİTİMİ

In [ ]:
from PIL import Image
import numpy as np

example_train_image = '/content/drive/MyDrive/melonamacancer/train/Benign/1.jpg'
image = Image.open(example_train_image)
# Görüntünün boyutunu kontrol etme
print("Görüntünün boyutu:", np.array(image).shape)



Görüntünün boyutu: (224, 224, 3)


In [ ]:
import os
from PIL import Image
import numpy as np

def load_images_with_labels(image_dir, label):
    X_images = []
    y_labels = []

    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Görüntü dosyalarını filtrele
            image_path = os.path.join(image_dir , filename)
            image = Image.open(image_path)
            # Görüntü boyutlarını ve kanal sayısını 28x28x3 olarak ayarla
            image = image.resize((28, 28))  # Örnek olarak 28x28 boyutuna yeniden boyutlandırma
            # Görüntüyü numpy dizisine dönüştür ve listeye ekle
            X_images.append(np.array(image))
            y_labels.append(label)  # Etiketi ekle

    return X_images, y_labels

# Benign görüntüleri yükle
X_train_benign, y_train_benign = load_images_with_labels('/content/drive/MyDrive/melonamacancer/train/Benign', 'Benign')

# Malignant görüntüleri yükle
X_train_malignant, y_train_malignant = load_images_with_labels('/content/drive/MyDrive/melonamacancer/train/Malignant', 'Malignant')

In [ ]:
# Tüm görüntüleri birleştir
X_train_images = X_train_benign + X_train_malignant
y_train_labels = y_train_benign + y_train_malignant

In [ ]:
def load_images_with_labels(image_dir, label):
    X_images = []
    y_labels = []

    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Görüntü dosyalarını filtrele
            image_path = os.path.join(image_dir , filename)
            image = Image.open(image_path)
            # Görüntü boyutlarını ve kanal sayısını 28x28x3 olarak ayarla
            image = image.resize((28, 28))  # Örnek olarak 28x28 boyutuna yeniden boyutlandırma
            # Görüntüyü numpy dizisine dönüştür ve listeye ekle
            X_images.append(np.array(image))
            y_labels.append(label)  # Etiketi ekle

    return X_images, y_labels

# Benign görüntüleri yükleme
X_test_benign, y_test_benign = load_images_with_labels('/content/drive/MyDrive/melonamacancer/test/Benign', 'Benign')

# Malignant görüntüleri yükleme
X_test_malignant, y_test_malignant = load_images_with_labels('/content/drive/MyDrive/melonamacancer/test/Malignant', 'Malignant')

In [ ]:
# Tüm görüntüleri birleştirme
X_test_images = X_test_benign + X_test_malignant
y_test_labels = y_test_benign + y_test_malignant

In [ ]:
y_test_labels

In [ ]:
X_train_images

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# X_train_images ve X_test_images listelerini numpy dizilerine dönüştürün
X_train_images = np.array(X_train_images)
X_test_images = np.array(X_test_images)

# Verileri yeniden şekillendirme ve normalleştirme
X_train_images = X_train_images.reshape(11930, 28, 28, 3).astype('float32') / 255.0
X_test_images = X_test_images.reshape(2020, 28, 28, 3).astype('float32') / 255.0

# Etiketleri sayısal değerlere dönüşümü
le = LabelEncoder()
y_train_labels = le.fit_transform(y_train_labels)
y_test_labels = le.transform(y_test_labels)

# Etiketleri one-hot encoding ile kodlama
y_train_labels = to_categorical(y_train_labels, num_classes=2)
y_test_labels = to_categorical(y_test_labels, num_classes=2)

In [ ]:
y_test_labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [ ]:
X_train_images

In [ ]:
# CNN modelini oluşturma
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')
])

# Modeli derleme
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitme
model.fit(X_train_images, y_train_labels, epochs=5, batch_size=1, validation_split=0.1)

# Modelin performansı
test_loss, test_acc = model.evaluate(X_test_images, y_test_labels)
print('Test accuracy:', test_acc)


Epoch 1/5
10737/10737 [==============================] - 61s 5ms/step - loss: 0.4239 - accuracy: 0.8065 - val_loss: 0.9262 - val_accuracy: 0.4677
Epoch 2/5
10737/10737 [==============================] - 52s 5ms/step - loss: 0.3552 - accuracy: 0.8433 - val_loss: 0.7022 - val_accuracy: 0.6597
Epoch 3/5
10737/10737 [==============================] - 53s 5ms/step - loss: 0.3339 - accuracy: 0.8552 - val_loss: 0.5168 - val_accuracy: 0.8013
Epoch 4/5
10737/10737 [==============================] - 53s 5ms/step - loss: 0.3131 - accuracy: 0.8636 - val_loss: 0.8051 - val_accuracy: 0.6521
Epoch 5/5
64/64 [==============================] - 1s 12ms/step - loss: 0.3132 - accuracy: 0.8619
Test accuracy: 0.8618811964988708


ANN'i SIFT ile besleme